# Pre-Processing for Model 2

In [1]:
import numpy as np
import json
import pandas as pd
from pandas.io.json import json_normalize
import xapian
import string
import sys
import time
import glob
from pandas import DataFrame
import spacy
import import_ipynb
from Features_extraction import *

importing Jupyter notebook from Features_extraction.ipynb


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
Did not use initialization regex that was passed: .*token_embedder_tokens\._projection.*weight
Did not use initialization regex that was passed: .*token_embedder_tokens\._projection.*weight


## Train Set pre_processing

In [ ]:
train_file = "traing_with_text.json"
train_set = pd.read_json(open(train_file),orient='index')

In [ ]:
train_set.count()

In [ ]:
train_set.head()

In [ ]:
support_data = train_set[train_set.label == "SUPPORTS"]             # search only Support
refute_data = train_set[train_set.label == "REFUTES"]               # search only Refutes

In [ ]:
n = 25000
support_data = support_data.sample(n)
refute_data = refute_data.sample(n)

In [ ]:
train_set = pd.concat([support_data,refute_data])

In [ ]:
train_set["text"] = train_set.text.apply(lambda x: " ".join(x))

In [ ]:
train_set.head()

## feature extraction

In [ ]:
%%time
agree = []
disagree = []
neutral=[]

    
for i, val in train_set.iterrows():
    claim = val['claim']
    evidence = val['text']
    
    score = allen_score(claim,evidence)
    a = score[0]
    dis = score[1]
    neu = score[2]
    agree.append(a)
    disagree.append(dis)
    neutral.append(neu)
    
train_set['agree_score'] = agree
train_set['disagree_score'] = disagree
train_set['neutral_score'] = neutral

In [ ]:
train_set.head()

In [ ]:
#adding column'response' where Support = 1 and refutes = 0
train_set['response'] = train_set['label'].apply(lambda x: 1 if x == 'SUPPORTS' else 0)
train_set.head()

In [ ]:
train_set.to_csv('train_with_new_features.csv') 

## Test Set pre_processing

In [ ]:
test_from_supriya = 'testing_respone_is_one.csv'
test_set_allen = pd.read_csv(open(test_from_supriya))
test_set_allen = test_set_allen.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)


In [69]:
def cleanItem(string):
    array = string.split(', ')
    return [str(array[0][2:-1]), int(array[1][1:-2])]

In [70]:
test_set_allen['evidence_topk']= test_set_allen['evidence_topk'].apply(lambda x:cleanItem(x))

In [71]:
test_set_with_text = test_set_allen.groupby(['claim_id','claim']).text_topk.apply(lambda x: ",".join(x)).reset_index()

In [72]:
test_set_with_text.head()

,claim_id,claim,text_topk
0,11,Carlos Santana formed Santana in 1966.,Santana is an American Latin rock band formed ...
1,24,The filming of Dilwale Dulhania Le Jayenge end...,Dilwale Dulhania Le Jayenge was one of only th...
2,69,David Packouz was born in 1982.,David Mordechai Packouz -LRB- -LSB- pækhaʊs -R...
3,77,Nicholas Brody is also called Nick.,"Nicholas `` Nick '' Brody , played by actor Da..."
4,83,Honeymoon is the third major-label record by L...,Honeymoon is the fourth studio album and third...


In [73]:
test_set_with_text = test_set_with_text.set_index('claim_id')
del test_set_with_text.index.name
test_set_with_text.head()

,claim,text_topk
11,Carlos Santana formed Santana in 1966.,Santana is an American Latin rock band formed ...
24,The filming of Dilwale Dulhania Le Jayenge end...,Dilwale Dulhania Le Jayenge was one of only th...
69,David Packouz was born in 1982.,David Mordechai Packouz -LRB- -LSB- pækhaʊs -R...
77,Nicholas Brody is also called Nick.,"Nicholas `` Nick '' Brody , played by actor Da..."
83,Honeymoon is the third major-label record by L...,Honeymoon is the fourth studio album and third...


In [74]:
test_set_with_pageid = test_set_allen.groupby(['claim_id','claim']).evidence_topk.apply(list).reset_index()

In [75]:
test_set_with_pageid = test_set_with_pageid.set_index('claim_id')
del test_set_with_pageid.index.name
test_set_with_pageid.head()

,claim,evidence_topk
11,Carlos Santana formed Santana in 1966.,"[[Santana_-LRB-band-RRB-, 0], [Carlos_Santana_..."
24,The filming of Dilwale Dulhania Le Jayenge end...,"[[Dilwale_Dulhania_Le_Jayenge, 15], [List_of_r..."
69,David Packouz was born in 1982.,"[[David_Packouz, 0]]"
77,Nicholas Brody is also called Nick.,"[[Nicholas_Brody, 1]]"
83,Honeymoon is the third major-label record by L...,"[[Honeymoon_-LRB-Lana_Del_Rey_album-RRB-, 0], ..."


In [76]:
test_set_with_pageid.rename(columns={"claim": "claim1"}, inplace= True)

In [77]:
test_set_with_evidence_text = test_set_with_pageid.join(test_set_with_text, how = 'left')
test_set_with_evidence_text = test_set_with_evidence_text.drop(['claim1'], axis=1)

In [78]:
test_set_with_evidence_text.count()

evidence_topk    9645
claim            9645
text_topk        9645
dtype: int64

In [79]:
test_set_with_evidence_text.head()

,evidence_topk,claim,text_topk
11,"[[Santana_-LRB-band-RRB-, 0], [Carlos_Santana_...",Carlos Santana formed Santana in 1966.,Santana is an American Latin rock band formed ...
24,"[[Dilwale_Dulhania_Le_Jayenge, 15], [List_of_r...",The filming of Dilwale Dulhania Le Jayenge end...,Dilwale Dulhania Le Jayenge was one of only th...
69,"[[David_Packouz, 0]]",David Packouz was born in 1982.,David Mordechai Packouz -LRB- -LSB- pækhaʊs -R...
77,"[[Nicholas_Brody, 1]]",Nicholas Brody is also called Nick.,"Nicholas `` Nick '' Brody , played by actor Da..."
83,"[[Honeymoon_-LRB-Lana_Del_Rey_album-RRB-, 0], ...",Honeymoon is the third major-label record by L...,Honeymoon is the fourth studio album and third...


## Feature Extraction

In [41]:
%%time
agree = []
disagree = []
neutral=[]

    
for i, val in test_set_with_evidence_text.iterrows():
    claim = val['claim']
    evidence = val['text_topk']
    
    score = allen_score(claim,evidence)
    a = score[0]
    dis = score[1]
    neu = score[2]
    agree.append(a)
    disagree.append(dis)
    neutral.append(neu)
    
test_set_with_evidence_text['agree_score'] = agree
test_set_with_evidence_text['disagree_score'] = disagree
test_set_with_evidence_text['neutral_score'] = neutral


CPU times: user 17h 37min 54s, sys: 37min 59s, total: 18h 15min 54s
Wall time: 2h 19min 29s


In [42]:
test_set_with_evidence_text.head()

,evidence_topk,claim,text_topk,agree_score,disagree_score,neutral_score
11,"[[Santana_-LRB-band-RRB-, 0], [Carlos_Santana_...",Carlos Santana formed Santana in 1966.,Santana is an American Latin rock band formed ...,0.999096,0.000896,0.000008
24,"[[Dilwale_Dulhania_Le_Jayenge, 15], [List_of_r...",The filming of Dilwale Dulhania Le Jayenge end...,Dilwale Dulhania Le Jayenge was one of only th...,0.998563,0.000712,0.000726
69,"[[David_Packouz, 0]]",David Packouz was born in 1982.,David Mordechai Packouz -LRB- -LSB- pækhaʊs -R...,0.393470,0.459105,0.147425
77,"[[Nicholas_Brody, 1]]",Nicholas Brody is also called Nick.,"Nicholas `` Nick '' Brody , played by actor Da...",0.133326,0.620620,0.246054
83,"[[Honeymoon_-LRB-Lana_Del_Rey_album-RRB-, 0], ...",Honeymoon is the third major-label record by L...,Honeymoon is the fourth studio album and third...,0.935128,0.064867,0.000005


In [ ]:
test_set_with_evidence_text.to_csv('test_with_new_features.csv')